In [8]:
import pandas as pd
import re
#import nltk
#from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
#from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
df = pd.read_csv('cleaned_data.csv')
dft = pd.read_csv('cleaned_data.csv')
df['skills'] = df[['skills']].fillna(value='unknown')
df['about'] = df[['about']].fillna(value='unknown')
df['name'] = df[['name']].fillna(value='unknown')
df['provider'] = df[['provider']].fillna(value='unknown')

dftx = pd.read_csv('dftx.csv')
tfidf = pd.concat([dft, dftx], axis=1)


dftx1 = pd.read_csv('dftx1.csv')
tfidf1 = pd.concat([dft, dftx1], axis=1)


dftx2 = pd.read_csv('dftx2.csv')
tfidf2 = pd.concat([dft, dftx2], axis=1)

In [10]:
#tf-idf  & cosine similarity

sim_a = cosine_similarity(dftx)
sim_n = cosine_similarity(dftx1)
sim_s = cosine_similarity(dftx2)
word_list_about = dftx.columns.get_values().tolist()
word_list_name = dftx1.columns.get_values().tolist()
word_list_skills = dftx2.columns.get_values().tolist()
dft['index']=dft.index

def tfidf_word(word):
    temp = df.copy()
    word = word.lower()
    
    if word in word_list_about:
        temp_a = tfidf.sort_values(by=word , ascending=False)[[word]]
        temp['result_a'] = temp_a[[word]]
    else:
        temp['result_a'] = 0.0
        
    if word in word_list_name:
        temp_n = tfidf1.sort_values(by=word , ascending=False)[[word]]
        temp['result_n'] = temp_n[[word]]
    else:
        temp['result_n'] = 0.0
    
    temp['result_weighted'] = 0.8 * temp['result_n'] + 0.2 * temp['result_a']
    temp = temp.sort_values(by=['result_weighted'] , ascending=False)
    #temp = temp[(temp['result_weighted'] > 0)]
    temp = temp.drop(columns=['result_a', 'result_n'])
    
    return temp

def tfidf_sim(index):
    dftx_temp = dft.copy()
    index = int(index)
    dftx_temp['sim_temp_about'] = sim_a[index]
    dftx_temp['sim_temp_name'] = sim_n[index]
    dftx_temp['sim_temp_skills'] = sim_s[index]
    dftx_temp['sim_result'] = 0.6 * dftx_temp['sim_temp_name'] + 0.2 * dftx_temp['sim_temp_about'] + 0.2 * dftx_temp['sim_temp_skills']
    dftx_temp = dftx_temp.sort_values(by="sim_result" , ascending=False)
    dftx_temp = dftx_temp.drop(columns=['category','sub-category','Unnamed: 0','sim_temp_about','sim_temp_name','sim_temp_skills'])
    dftx_temp = dftx_temp.drop_duplicates(subset='name', keep="first")
    dftx_temp = dftx_temp[1:6]
    dftx_temp = dftx_temp[(dftx_temp['sim_result'] > 0)]
    return dftx_temp

def tfidf_words(sentence):
    wordlist = re.sub("[^\w]", " ",  sentence).split()
    temp = pd.DataFrame()
    for i in range(len(wordlist)):
        temp[wordlist[i]] = tfidf_word(wordlist[i])['result_weighted']
    
    temp.loc[:,'Total'] = temp.sum(axis=1)
    temp = temp[['Total']]
    
    res = pd.concat([dft, temp], axis=1).sort_values(by="Total" , ascending=False)
    res = res.drop_duplicates(subset='name', keep="first")
    res= res.drop(columns=['category','sub-category','Unnamed: 0'])
    res = res[(res['Total'] > 0)]
    return res

In [11]:
tfidf_words('big data')

,name,provider,overall,enrolled,about,subtitles,skills,url,topic,index,Total
276,"Big Data, Genes, and Medicine",The State University of New York,4.2,14885,"10,891 This course distills for you expert kn...",English,Bioinformatics Data Clustering Algorithms Big...,https://www.coursera.org/learn/data-genes-medi...,Health,276,0.850811
320,Big History - From the Big Bang until Today,University of Amsterdam,4.6,16266,"5,154 Welcome to this Big History course! In ...","English, Spanish",unknown,https://www.coursera.org/learn/bighistory,Nature Science,320,0.611095
132,Data Visualization,University of Illinois at Urbana-Champaign,4.5,62678,"47,225 Learn the general concepts of data min...","English, Korean",Data Visualization Software Tableau Software ...,https://www.coursera.org/learn/datavisualization,Math and Computer Science,132,0.478176
20,Big History: Connecting Knowledge,Macquarie University,4.8,47879,"38,733 We currently face unprecedented challe...","English, Catalan, Romanian",unknown,https://www.coursera.org/learn/big-history,"Language, Arts and Social",20,0.456053
125,Getting and Cleaning Data,Johns Hopkins University,4.6,119493,"119,579 Before you can work with data you hav...","French, Chinese (Simplified), Vietnamese, Russ...",Data Manipulation Regular Expression (REGEX) ...,https://www.coursera.org/learn/data-cleaning,Math and Computer Science,125,0.453390
90,Introduction to Data Science in Python,University of Michigan,4.5,270492,"727,986 This course will introduce the learne...","Chinese (Traditional), Portuguese (Brazilian),...",Python Programming Numpy Pandas Data Cleansing,https://www.coursera.org/learn/python-data-ana...,Math and Computer Science,90,0.441453
136,Developing Data Products,Johns Hopkins University,4.6,43857,"49,445 A data product is the production outpu...",English,Interactivity Plotly Web Application R Progra...,https://www.coursera.org/learn/data-products,Math and Computer Science,136,0.426520
145,Introduction to Probability and Data,Duke University,4.7,134075,"204,691 This course introduces you to samplin...","English, Korean",Statistics R Programming Rstudio Exploratory ...,https://www.coursera.org/learn/probability-intro,Math and Computer Science,145,0.422411
139,Python Data Structures,University of Michigan,4.9,288393,"712,240 This course will introduce the core d...","English, Korean, Arabic",Python Syntax And Semantics Data Structure Tu...,https://www.coursera.org/learn/python-data,Math and Computer Science,139,0.417093
73,A Crash Course in Data Science,Johns Hopkins University,4.5,117206,"45,812 By now you have definitely heard about...","Chinese (Traditional), Arabic, Russian, Turkis...",Data Science Data Analysis Machine Learning P...,https://www.coursera.org/learn/data-science-co...,Business,73,0.409121
